In [82]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.utils import np_utils
import keras
import numpy as np

np.random.seed(5)

In [83]:
class LossHistory(keras.callbacks.Callback):
  def init(self):
    self.losses = []

  def on_epoch_end(self, batch, logs={}):
    self.losses.append(logs.get('loss'))

In [84]:
code2idx = {'c4':0, 'd4':1, 'e4':2, 'f4':3, 'g4':4, 'a4':5, 'b4':6,
            'c8':7, 'd8':8, 'e8':9, 'f8':10, 'g8':11, 'a8':12, 'b8':13}

idx2code = {0:'c4', 1:'d4', 2:'e4', 3:'f4', 4:'g4', 5:'a4', 6:'b4',
            7:'c8', 8:'d8', 9:'e8', 10:'f8', 11:'g8', 12:'a8', 13:'b8'}

In [85]:
def seq2dataset(seq, window_size):
    dataset = []
    for i in range(len(seq)-window_size):
        subset = seq[i:(i+window_size+1)]
        dataset.append([code2idx[item] for item in subset])
    return np.array(dataset)

In [86]:
seq = ['g8', 'e8', 'e4', 'f8', 'd8', 'd4', 'c8', 'd8', 'e8', 'f8', 'g8', 'g8', 'g4',
       'g8', 'e8', 'e8', 'e8', 'f8', 'd8', 'd4', 'c8', 'e8', 'g8', 'g8', 'e8', 'e8', 'e4',
       'd8', 'd8', 'd8', 'd8', 'd8', 'e8', 'f4', 'e8', 'e8', 'e8', 'e8', 'e8', 'f8', 'g4',
       'g8', 'e8', 'e4', 'f8', 'd8', 'd4', 'c8', 'e8', 'g8', 'g8', 'e8', 'e8', 'e4']

dataset = seq2dataset(seq, window_size = 4)

In [87]:
# x_train은 전체 길이 중 0~3번째 까지 / y_train은 3번째 것만
x_train = dataset[:, 0:4]
y_train = dataset[:, 4]

max_idx_value = 13

x_train = x_train / float(max_idx_value)
x_train = np.reshape(x_train, (50, 4, 1))

y_train = np_utils.to_categorical(y_train)

one_hot_vec_size = y_train.shape[1]

In [88]:
model = Sequential()
model.add(LSTM(128, input_shape=(4, 1)))
model.add(Dense(one_hot_vec_size, activation='softmax'))

In [89]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_14 (LSTM)               (None, 128)               66560     
_________________________________________________________________
dense_13 (Dense)             (None, 12)                1548      
Total params: 68,108
Trainable params: 68,108
Non-trainable params: 0
_________________________________________________________________


In [90]:
history = LossHistory()
history.init()

In [91]:
model.fit(x_train, y_train, epochs=2000, batch_size=4, verbose=2, callbacks=[history])
# 최종 accuracy가 90프로 정도임

Epoch 1/2000
 - 0s - loss: 2.4435 - accuracy: 0.2800
Epoch 2/2000
 - 0s - loss: 2.3205 - accuracy: 0.3400
Epoch 3/2000
 - 0s - loss: 2.1610 - accuracy: 0.3400
Epoch 4/2000
 - 0s - loss: 2.0287 - accuracy: 0.3400
Epoch 5/2000
 - 0s - loss: 1.9826 - accuracy: 0.3400
Epoch 6/2000
 - 0s - loss: 1.9570 - accuracy: 0.3400
Epoch 7/2000
 - 0s - loss: 1.9271 - accuracy: 0.3400
Epoch 8/2000
 - 0s - loss: 1.9182 - accuracy: 0.3400
Epoch 9/2000
 - 0s - loss: 1.9127 - accuracy: 0.3400
Epoch 10/2000
 - 0s - loss: 1.9200 - accuracy: 0.3400
Epoch 11/2000
 - 0s - loss: 1.9133 - accuracy: 0.3400
Epoch 12/2000
 - 0s - loss: 1.9094 - accuracy: 0.3400
Epoch 13/2000
 - 0s - loss: 1.9111 - accuracy: 0.3400
Epoch 14/2000
 - 0s - loss: 1.9119 - accuracy: 0.3400
Epoch 15/2000
 - 0s - loss: 1.9070 - accuracy: 0.3400
Epoch 16/2000
 - 0s - loss: 1.8964 - accuracy: 0.3400
Epoch 17/2000
 - 0s - loss: 1.9048 - accuracy: 0.3400
Epoch 18/2000
 - 0s - loss: 1.8972 - accuracy: 0.3400
Epoch 19/2000
 - 0s - loss: 1.8920 - 

In [97]:
# 여기부터 상태유지 LSTM을 활용함 -> 학습 상태가 다음 샘플에 그대로 전달

model = Sequential()
model.add(LSTM(128, batch_input_shape=(1,4,1), stateful='True')) #stateful='True'핵심
model.add(Dense(one_hot_vec_size, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [100]:
num_epochs = 100

# 샘플 학습을 마치고 새로운 샘플 학습을 위해 상태 초기화
for epoch_idx in range(num_epochs):
    print ('epochs : ' + str(epoch_idx) )
    model.fit(x_train, y_train, epochs=1, batch_size=1, verbose=2, shuffle=False, callbacks=[history]) # 50 is X.shape[0]
    model.reset_states()

epochs : 0
Epoch 1/1
 - 0s - loss: 0.6595 - accuracy: 0.7600
epochs : 1
Epoch 1/1
 - 0s - loss: 0.4992 - accuracy: 0.8600
epochs : 2
Epoch 1/1
 - 0s - loss: 0.9443 - accuracy: 0.7000
epochs : 3
Epoch 1/1
 - 0s - loss: 0.3532 - accuracy: 0.9200
epochs : 4
Epoch 1/1
 - 0s - loss: 0.2190 - accuracy: 0.9800
epochs : 5
Epoch 1/1
 - 0s - loss: 0.3573 - accuracy: 0.9400
epochs : 6
Epoch 1/1
 - 0s - loss: 0.3465 - accuracy: 0.8800
epochs : 7
Epoch 1/1
 - 0s - loss: 0.6131 - accuracy: 0.8800
epochs : 8
Epoch 1/1
 - 0s - loss: 0.4936 - accuracy: 0.7800
epochs : 9
Epoch 1/1
 - 0s - loss: 0.3630 - accuracy: 0.8400
epochs : 10
Epoch 1/1
 - 0s - loss: 0.2776 - accuracy: 0.9000
epochs : 11
Epoch 1/1
 - 0s - loss: 0.2178 - accuracy: 0.9000
epochs : 12
Epoch 1/1
 - 0s - loss: 0.1531 - accuracy: 0.9600
epochs : 13
Epoch 1/1
 - 0s - loss: 0.0771 - accuracy: 1.0000
epochs : 14
Epoch 1/1
 - 0s - loss: 0.0538 - accuracy: 1.0000
epochs : 15
Epoch 1/1
 - 0s - loss: 0.0412 - accuracy: 1.0000
epochs : 16
Epoch 

In [77]:
scores = model.evaluate(x_train, y_train, batch_size=1)
print(scores)
print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))
model.reset_states()

50/50 [==============================] - 0s 1ms/step
[0.001270484580163611, 1.0]
accuracy: 100.00%
